In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import time

In [ ]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df.head()

In [ ]:
train_df.info()
test_df.info()

In [ ]:
test_df.head()

In [ ]:
train_df.describe()

In [ ]:
train_df['target'].value_counts().plot(kind='bar', label='target')
plt.legend()
plt.title('Target distribution')

In [ ]:
def plot_feature_distribution(df1, df2, label1, label2, features):
    i = 0
    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(10,10,figsize=(18,22))

    for feature in features:
        i += 1
        plt.subplot(10,10,i)
        sns.kdeplot(df1[feature], bw=0.5,label=label1)
        sns.kdeplot(df2[feature], bw=0.5,label=label2)
        plt.xlabel(feature, fontsize=9)
        locs, labels = plt.xticks()
        plt.tick_params(axis='x', which='major', labelsize=6, pad=-6)
        plt.tick_params(axis='y', which='major', labelsize=6)
    plt.show();

In [ ]:
t0 = train_df.loc[train_df['target'] == 0]
t1 = train_df.loc[train_df['target'] == 1]
features = train_df.columns.values[2:102]
plot_feature_distribution(t0, t1, '0', '1', features)

In [ ]:
t0 = train_df.loc[train_df['target'] == 0]
t1 = train_df.loc[train_df['target'] == 1]
features = train_df.columns.values[102:202]
plot_feature_distribution(t0, t1, '0', '1', features)

In [ ]:
corr_matrix = train_df.drop(['ID_code', 'target'], axis=1).corr().abs().unstack().sort_values(kind="quicksort").reset_index()
corr_matrix = corr_matrix[corr_matrix["level_0"] != corr_matrix["level_1"]]

In [ ]:
corr_matrix.tail()

In [ ]:
cols=["target","ID_code"]
X = train_df.drop(cols,axis=1)
y = train_df["target"]
X_test  = test_df.drop("ID_code",axis=1)

In [ ]:
params = {'objective' : "binary", 
          'is_unbalance': True,
               'boost':"dart",
               'metric':"auc",
               'boost_from_average':"false",
               'num_threads':0,
               'learning_rate' : 0.01,
               'num_leaves' : 23,
               'max_depth':-1,
               'feature_fraction' : 0.1,
               'bagging_freq' : 5,
               'bagging_fraction' : 0.4,
               'min_data_in_leaf' : 100,
               'min_sum_hessian_in_leaf' : 10.0,
               'verbosity' : 1}
fold_n=5
folds = StratifiedKFold(n_splits=fold_n, shuffle=True, random_state=10)

In [ ]:
%%time
y_pred_lgb = np.zeros(len(X_test))
num_round = 1000000
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    lgb_model = lgb.train(params,train_data,num_round,#change 20 to 2000
                    valid_sets = [train_data, valid_data],verbose_eval=1000,early_stopping_rounds = 3500)##change 10 to 200
            
    y_pred_lgb += lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)/5

In [ ]:
sub_df = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df["target"] = prediction[:,1]
sub_df.to_csv("submission.csv", index=False)
